In [4]:
import numpy as np
import time
import copy
from scipy.optimize import differential_evolution

In [5]:
nSamples = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]

In [3]:
# p - persistance
# r - ready time
# d - oczekiwany termin zakończenia
# w - weight
# Cj oznacza moment zakończenia wykonywania zadania Jj w uszeregowaniu

# p r d w
# r  >= c - p

# Generate

In [4]:
for x in nSamples:
    time = 0
    fileData = []
    fileData.append(x)
    for _ in range(x):
        p = np.random.randint(1, 10)
        r = np.random.randint(time - 5 if time - 5 > 0 else 0 ,time + 5)
        d = np.random.randint(r+p, r+p+10)
        w = np.random.randint(0,10)
        time += p
        
        fileData.append([p,r,d,w])
    with open(f'data/instances_{x}.csv', "w") as f:
        f.write(f"{str(fileData[0])}\n")
        for line in fileData[1:]:            
            f.write(f"{' '.join([str(x) for x in line])}\n")

NameError: name 'n' is not defined

# Verify

In [5]:
x = 50

In [7]:
def calculateCost(data, taskQueue):
    calculatedWeight = 0
    timeCounter = 0
    for task in taskQueue:
        if timeCounter < data[task]['r']:
            timeCounter = data[task]['r']
        # add p time to current time
        timeCounter += data[task]['p']
        c = timeCounter
        data[task]['c'] = c

        # check correctness
        if not data[task]["r"] <= data[task]["c"] - data[task]["p"]: 
            print("Error")

        # verify cost
        if data[task]["c"] > data[task]["d"]:
            calculatedWeight += data[task]["w"]
    #print(calculatedWeight)
    return calculatedWeight

In [27]:
motherFolder = "out" # "PTSZ-instancje"
data = {}
indicies = ["136805", "136792", "136683", "132231", "136730", "136682", "136764", "136782", "136723", "136778", "136309", "136718", "136315", "136759"]
for index  in indicies:
    print(index)
    for samples in nSamples:
        with open(f'{motherFolder}/{index}/out{index}_{samples}.txt', "r") as result:
            res = result.read().split('\n')
            weight = int(res[0])
            gen = (t for t in res[1].split(" ") if t.isdigit())
            taskQueue = [int(t) for t in gen]

        with open(f'PTSZ-instancje/{index}/in{index}_{samples}.txt', "r") as instance:
            d = instance.read().split('\n')
            n = int(d[0])                
            for i,line in enumerate(d[1:]):
                if line == '':
                    continue
                p,r,d,w = line.split(' ')
                p,r,d,w = int(p), int(r), int(d), int(w)
                data[i+1] = {"p": p, "r": r, "d": d, "w": w, "c": 0 }
                    
        calculatedWeight = calculateCost(data, taskQueue)
        print(calculatedWeight)
        #if calculatedWeight != weight:
        #    print(f"wrong weight. Calculated: {calculatedWeight}, read: {weight}")
        #else:
        #    print(f"Weight: {calculatedWeight} confirmed")
    print()
    print()
        
        

136805
51
135
215
245
319
377
445
503
599
503


136792
97
420
1012
1877
2793
4166
5806
8022
10249
11932


136683
219
931
2202
3871
5797
8849
11986
16473
20214
24833


132231
244
534
844
1147
1363
1652
1860
2189
2494
2805


136730
0
0
256
540
833
1095
1351
1759
2200
2477


136682
36
147
376
685
1722
1250
1838
2235
3441
3793


136764
88
303
688
1316
2379
3301
4393
5560
7403
10111


136782
195
427
694
943
1189
1451
1596
1917
2086
2304


136723
55
86
187
232
322
386
415
560
540
631


136778
115
271
373
425
562
776
807
1039
1101
1186


136309


KeyError: 0

In [11]:
# p - persistance
# r - ready time
# d - oczekiwany termin zakończenia
# w - weight
# Cj oznacza moment zakończenia wykonywania zadania Jj w uszeregowaniu

# p r d w
# r  >= c - p

# TaskScheduler

In [5]:
a,b,c,d = (-140036.8041378972, 2.5486165590751364, -0.1998934445512417, 1183)
totalCost = 0
data = {}
indicies = ["136805", "136792", "136683", "132231", "136730", "136682", "136764", "136782", "136723", "136778", "136309", "136718", "136315", "136759"]
for index  in indicies:
    for samples in nSamples:
        with open(f'PTSZ-instancje/{index}/in{index}_{samples}.txt', "r") as instance:
            d = instance.read().split('\n')
            n = int(d[0])                
            for i,line in enumerate(d[1:]):
                if line == '':
                    continue
                p,r,d,w = line.split(' ')
                p,r,d,w = int(p), int(r), int(d), int(w)
                data[i+1] = {"p": p, "r": r, "d": d, "w": w, "c": 0, "penalty": 0 }
            dataCopy = copy.deepcopy(data)
        
                    
        calculatedWeight = 0
        timeCounter = 0
        outSchedule = []
        
        start = time.time()
        while len(data) > 0:
            readyToGo = {}
            for key, task in data.items():
                if task['r'] <= timeCounter: # add penalty ( (timeCounter+p-d)*w )
                    endTime = timeCounter+task['p']-task['d']
                    penaltyEndTask = ( endTime if endTime > 0 else 0 ) * task['w']
                    penaltyWeight = task['w']
                    penaltyPersistance = task['p']
                    penaltyDue = task['d'] - timeCounter
                    task['penalty'] = a*penaltyEndTask + b*penaltyWeight + c*penaltyPersistance - d*penaltyDue
                    readyToGo[key] = task
                
            if len(readyToGo) == 0: # currently there is no task to go
                #print("upsi")
                timeCounter += 1
                continue
            #print(f"readyToGo: {readyToGo}")

            readyToGoSorted = sorted(readyToGo.items(), key=lambda y: (y[1]['penalty'], y[1]['w']), reverse=True)
            selectedTask = readyToGoSorted.pop(0)
            #print(timeCounter)
            #print(f"selectedTask: {selectedTask}")

            data.pop(selectedTask[0])
            timeCounter += selectedTask[1]['p']
            outSchedule.append(selectedTask[0])    
        
        end = time.time()
        #print(end-start)
        
        cost = calculateCost(dataCopy, outSchedule)
        #print(cost)
        totalCost += cost
        
        with open(f'out/{index}/out{index}_{samples}.txt', "w") as f:
            f.write(f"{cost}\n")         
                #f.write(f"{' '.join([str(x) for x in line])}\n")
            f.write(f"{' '.join([str(x) for x in outSchedule])}\n")
        
print(totalCost)

385570


In [28]:
497400

501021

In [9]:
from scipy.optimize import minimize

In [10]:
x0 = (1, 1, 1, 1)

In [13]:
def costFunction(x):
    a,b,c,d = x
    totalCost = 0
    data = {}
    indicies = ["136805", "136792", "136683", "132231", "136730", "136682", "136764", "136782", "136723", "136778", "136309", "136718", "136315", "136759"]
    for index  in indicies:
        for samples in nSamples:
            with open(f'PTSZ-instancje/{index}/in{index}_{samples}.txt', "r") as instance:
                d = instance.read().split('\n')
                n = int(d[0])                
                for i,line in enumerate(d[1:]):
                    if line == '':
                        continue
                    p,r,d,w = line.split(' ')
                    p,r,d,w = int(p), int(r), int(d), int(w)
                    data[i+1] = {"p": p, "r": r, "d": d, "w": w, "c": 0, "penalty": 0 }
                dataCopy = copy.deepcopy(data)


            calculatedWeight = 0
            timeCounter = 0
            outSchedule = []

            start = time.time()
            while len(data) > 0:
                readyToGo = {}
                for key, task in data.items():
                    if task['r'] <= timeCounter: # add penalty ( (timeCounter+p-d)*w )
                        endTime = timeCounter+task['p']-task['d']
                        penaltyEndTask = ( endTime if endTime > 0 else 0 ) * task['w']
                        penaltyWeight = task['w']
                        penaltyPersistance = task['p']
                        penaltyDue = task['d'] - timeCounter
                        task['penalty'] = a*penaltyEndTask + b*penaltyWeight + c*penaltyPersistance + d*penaltyDue
                        readyToGo[key] = task

                if len(readyToGo) == 0: # currently there is no task to go
                    #print("upsi")
                    timeCounter += 1
                    continue
                #print(f"readyToGo: {readyToGo}")

                readyToGoSorted = sorted(readyToGo.items(), key=lambda y: (y[1]['penalty'], y[1]['w']), reverse=True)
                selectedTask = readyToGoSorted.pop(0)
                #print(timeCounter)
                #print(f"selectedTask: {selectedTask}")

                data.pop(selectedTask[0])
                timeCounter += selectedTask[1]['p']
                outSchedule.append(selectedTask[0])    

            end = time.time()
          #  print(int((end-start)*1000) if int((end-start)*1000) != 0 else 1)

            cost = calculateCost(dataCopy, outSchedule)
            #print(cost)
            totalCost += cost
    print(f"[{totalCost},{a},{b},{c},{d}],")
    return totalCost

In [18]:
options = {"maxiter":40, "disp":True}

In [10]:

res = minimize(costFunction, x0, method='TNC', options=options, tol=1e-2)

NameError: name 'minimize' is not defined

In [16]:
bounds = [(-20000,20000), (-20000, 20000), (-20000, 20000), (-20000, 20000)]

In [17]:
res = differential_evolution(costFunction,bounds)

[395167,-19324.778169070774,6460.475127805929,-12177.983836703328,1183],
[442982,-16844.610798662925,-14364.912568584183,18771.001383141433,1183],
[499914,5580.863862917434,11302.174026694125,-14633.884679346991,1183],
[426830,-17764.738526628284,-12826.821634528415,-13150.541054662786,1183],
[440170,-14823.535071921595,-19600.834763637922,-4450.905682077082,1183],
[427206,-1979.1108857541806,-6532.114078959464,-8527.259647698615,1183],
[502041,12378.815971127253,7823.744589735151,-5003.95059731759,1183],
[500926,11322.074772822574,15547.97527246925,-11500.33526975001,1183],
[497649,1794.7142867317866,10458.58312293901,163.18972990923885,1183],
[498201,3429.0573461083573,16395.251645086653,-6578.03436213217,1183],
[507385,18847.84291283751,-4744.439682206709,10310.924864504506,1183],
[507131,13098.357433027688,-3852.708950264139,8729.322280059547,1183],
[433711,-3334.314580088087,-11348.971895137083,-7810.883352460672,1183],
[423784,-15572.875981280951,9782.726970021147,11905.256167837

[411555,-7084.920393729081,18070.107216280623,1097.46015169927,1183],
[425065,-10994.004509206488,-13378.283258008903,-16331.129862737984,1183],
[429353,-7072.37862794607,-8557.883489046251,-7956.214227095226,1183],
[392117,-16498.24865457309,15168.989980074517,-13132.247324136977,1183],
[414875,-11580.641156679827,19972.703460718043,6376.397626738158,1183],
[422142,-16498.24865457309,-8185.837229966517,-13132.247324136977,1183],
[461410,146.24687475878417,19756.805935366665,-768.8767002267971,1183],
[391626,-3150.799735678671,18922.713419299413,-14633.884679346991,1183],
[425637,-12682.573879517251,-12826.821634528415,-15073.742326145313,1183],
[395316,-14823.535071921595,9426.522662740093,-9953.72470380902,1183],
[391469,-19085.767442200024,10771.867462245575,-15366.734803137819,1183],
[503696,8748.247142324894,-1515.4937407696423,-12430.813884856807,1183],
[500887,11322.074772822574,15547.97527246925,-13228.92379631078,1183],
[420623,-5026.395181659069,10458.58312293901,6641.1711956

[390652,-12664.251423613981,14120.273357799124,-15572.333592083352,1183],
[398452,-13726.073247036575,19143.547694365105,-5711.935039491034,1183],
[413382,-13812.425538956659,18314.66587008504,2708.8884055185367,1183],
[502136,16932.473884118146,14545.41681757167,-2695.955892889774,1183],
[506517,12406.93391414507,-1656.0620681122361,10797.727941421665,1183],
[436562,-13771.725181814094,-1270.4197462036993,6823.82034890284,1183],
[426077,-10324.139980093063,-13642.40897156276,-14913.351375736607,1183],
[391614,-7249.338622073076,15508.444663426011,-14365.8059248488,1183],
[395152,-1542.1783550410373,11202.273175870707,-10848.488234278828,1183],
[388913,-16685.611907630402,13153.347983783493,-19975.20077923287,1183],
[390433,-7692.9121340463835,10494.42178321283,-19473.183272995215,1183],
[389951,-17406.34243852013,18866.156956726973,-15699.467284370314,1183],
[437073,-4765.6370114098,-14030.733210798791,-6038.664612414704,1183],
[389337,-19406.944512259117,15168.989980074517,-17000.697

[390869,-13765.061883762888,11227.095511436244,-16492.12975024591,1183],
[419084,-16262.904916735006,16133.585054070072,11642.27149790504,1183],
[399369,-18467.91638253612,14792.347996077888,-5394.400310791181,1183],
[390327,-12030.793058937934,18465.214394163497,-15778.29562703271,1183],
[419040,-13224.13968486059,-6561.384233882625,-14434.177712156756,1183],
[388421,-19649.694222837472,19658.184403858002,-17775.931233250023,1183],
[413541,-3110.2272323565617,14120.273357799124,919.7531327353125,1183],
[425014,-19290.13134947397,-14467.295760113833,-17012.432619659616,1183],
[389518,-9432.633698103538,19986.065167295405,-17033.466888064642,1183],
[394554,-8918.128944179483,19478.583063657938,-9176.629802483098,1183],
[397284,-18456.776058940533,17719.426605757024,-6999.34413510042,1183],
[415202,-16086.071094273682,19105.129400449532,6823.82034890284,1183],
[396348,-19551.121520370467,11342.495501663041,-8914.702109445352,1183],
[391047,-14604.403191608735,15508.444663426011,-14365.80

[443401,-9318.263090506145,-12080.74030261308,14641.325770443098,1183],
[440972,-16714.54386305788,-7677.064889127742,1964.9493348591163,1183],
[387904,-12356.059062397571,19262.14246371558,-19703.083747500947,1183],
[430461,-2567.744441132933,-11331.33565751451,-11276.423476186754,1183],
[396434,-15235.614525064793,14663.709594304173,-8157.392148301734,1183],
[420782,-10880.851798896301,5095.395461294023,1395.0659708868818,1183],
[416881,-19903.05293369795,19376.857560319124,9895.94145989912,1183],
[401322,-14606.568227348498,19251.30139747238,-3686.133326831187,1183],
[391148,-14745.260995527879,9752.510324536319,-17560.833944121947,1183],
[388783,-13707.850118630715,14792.347996077888,-19310.10693713287,1183],
[389207,-12993.551422910348,13606.832947831932,-19810.694659565346,1183],
[507579,15248.139719803046,-15424.36395687417,-8863.759724966872,1183],
[387963,-17160.24669362616,19761.3770326,-19642.571939319863,1183],
[391807,-9657.589823017068,9656.524051019564,-16645.7433903413,

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x = [1,2,3]

In [ ]:
plt.plot(x)
plt.title("Minimalizacja funkcji celu")
plt.xaxis("iteracja")
plt.yaxis("Koszt")

In [94]:
costFunction(x)

1
7
13
25
17
19
26
33
38
61
1
4
10
17
29
42
64
85
99
140
1
4
11
19
30
45
63
152
111
133
1
3
15
16
25
32
48
61
82
88
1
1
6
13
30
36
48
64
94
108
1
3
6
11
19
27
38
52
62
79
1
3
8
22
29
38
56
71
105
124
1
4
10
17
26
39
53
69
105
107
1
1
1
2
2
4
7
6
9
10
1
3
6
10
16
26
33
54
57
68
1
3
7
15
22
45
45
56
75
108
1
1
1
1
2
4
5
7
8
11
1
4
11
21
32
47
74
87
110
146
1
3
5
8
14
28
34
39
54
84


385570

In [85]:
x = (-140036.8041378972, 2.5486165590751364, -0.1998934445512417, 1183)